In [1]:
import os
import sys
from pprint import pprint
from dotenv import load_dotenv
import pandas as pd
import numpy as np
#import seaborn as sns
#import time
import xgboost as xgb
from xgboost import plot_tree
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import operator
import random
import graphviz
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#conda install -c conda-forge bayesian-optimization
from bayes_opt import BayesianOptimization
import pandas_profiling
load_dotenv('forecast_arosa.env')
random.seed(22)

# Open CSV


In [2]:
path_in = os.getenv('DATAIN') # this is where we downloaded the tabel

In [3]:
file_name = os.getenv('DATASET_FILENAME') # this is where we downloaded the tabel

In [4]:
os.path.join(path_in, file_name)

'../arosa_data/arosa_data_feature1.csv'

In [5]:
df = pd.read_csv(os.path.join(path_in, file_name),delimiter=';')
df

,Datum,Wochentag,TagNr,WocheNr,isFreiZH,isFreiSG,isFreiGR,isFerienZH,Ferien Desc ZH,IsFerienZH_plus3,...,precip_1h_mm_sum,fresh_snow_1h_cm_sum,wind_speed_mean_10m_1h_ms_avg,weather_symbol_1h_idx_spe,number_of_events,isHumorfestival,Schalter,Tel,Mail,Total Anfragen
0,01.01.2016,Freitag,1,53,1,1,1,0,NaN,0,...,0.81,1.3,2.6,1.1,0,0,10,20,5,35
1,02.01.2016,Samstag,2,53,1,1,1,0,NaN,0,...,1.21,1.9,5.4,0.0,0,0,78,59,22,159
2,03.01.2016,Sonntag,3,53,1,1,1,0,NaN,0,...,0.23,0.0,5.9,0.0,0,0,0,0,0,0
3,04.01.2016,Montag,4,1,0,0,0,0,NaN,0,...,0.13,0.3,2.4,0.0,0,0,0,0,0,0
4,05.01.2016,Dienstag,5,1,0,0,0,0,NaN,0,...,0.00,0.0,3.4,1.3,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334,27.08.2019,Dienstag,1335,35,0,0,0,0,NaN,0,...,0.00,0.0,11.2,0.0,2,0,59,26,29,114
1335,28.08.2019,Mittwoch,1336,35,0,0,0,0,NaN,0,...,0.00,0.0,5.6,1.1,2,0,61,40,19,120
1336,29.08.2019,Donnerstag,1337,35,0,0,0,0,NaN,0,...,0.16,0.0,5.8,0.0,2,0,76,38,23,137
1337,30.08.2019,Freitag,1338,35,0,0,0,0,NaN,0,...,0.00,0.0,5.7,0.7,3,0,157,48,27,232


In [6]:
df[df['Datum']=='31.08.2019']

,Datum,Wochentag,TagNr,WocheNr,isFreiZH,isFreiSG,isFreiGR,isFerienZH,Ferien Desc ZH,IsFerienZH_plus3,...,precip_1h_mm_sum,fresh_snow_1h_cm_sum,wind_speed_mean_10m_1h_ms_avg,weather_symbol_1h_idx_spe,number_of_events,isHumorfestival,Schalter,Tel,Mail,Total Anfragen
1338,31.08.2019,Samstag,1339,35,1,1,1,0,NaN,0,...,0.03,0.0,5.8,2.9,3,0,183,33,23,239


In [16]:
len(df)

1339

In [17]:
df= df[df.Datum != '09.03.2016']

# Overview of Data

In [8]:
#df.profile_report()

# Preprocessing Error

In [20]:
list = ['Datum',
 'Wochentag',
 'season',
 'isFerienZH',
 'Ferien Desc ZH',
 'isFerienSG',
 'Ferien Desc SG',
 'isFerienGR',
 'Ferien Desc GR',
 'Feiertag Desc',
 'isFeiertag ZH',
 'isFeiertag SG',
 'isFeiertag GR',
 't_2m_c_avg',
 'snow_depth_cm_avg',
 'visibility_m_avg',
 'wind_speed_10m_ms_max',
 'prob_precip_1h_p_avg',
 'prob_tstorm_1h_p_avg',
 'hail_idx_avg',
 'wind_gusts_10m_ms_max',
 'wind_speed_mean_10m_24h_ms_avg',
 't_max_2m_24h_c_max',
 't_min_2m_24h_c_min',
 'precip_24h_mm_max',
 'fresh_snow_24h_cm_max',
 'is_sleet_1h_idx_max',
 'is_fog_1h_idx_max',
 'precip_1h_mm_sum',
 'fresh_snow_1h_cm_sum',
 'wind_speed_mean_10m_1h_ms_avg',
 'weather_symbol_1h_idx_spe',
 'number_of_events',
 'Schalter',
 'Tel',
 'Mail',
 'Total Anfragen']
df
df[list]

KeyError: "['season'] not in index"

## LabelEncoder 

In [169]:
df['Ferien Desc ZH'].fillna(value=0, inplace = True)
df['Ferien Desc GR'].fillna(value=0, inplace = True)
df['Ferien Desc SG'].fillna(value=0, inplace = True)
df['Feiertag Desc'].fillna(value=0, inplace = True)

In [170]:
df = df.astype({"Ferien Desc ZH": str, "Ferien Desc GR": str, "Ferien Desc SG": str, "Feiertag Desc": str})

In [171]:
df['Ferien Desc ZH'].value_counts()

0                   1024
Sommerferien         145
Sportferien           64
Fruehlingsferien      61
Herbstferien          45
Name: Ferien Desc ZH, dtype: int64

In [172]:
df=df.drop(['Datum'],axis=1)
df

,Wochentag,TagNr,WocheNr,isFreiZH,isFreiSG,isFreiGR,isFerienZH,Ferien Desc ZH,IsFerienZH_plus3,isFerienZH_minus3,...,precip_1h_mm_sum,fresh_snow_1h_cm_sum,wind_speed_mean_10m_1h_ms_avg,weather_symbol_1h_idx_spe,number_of_events,isHumorfestival,Schalter,Tel,Mail,Total Anfragen
0,Freitag,1,53,1,1,1,0,0,0,0,...,0.81,1.3,2.6,1.1,0,0,10,20,5,35
1,Samstag,2,53,1,1,1,0,0,0,0,...,1.21,1.9,5.4,0.0,0,0,78,59,22,159
2,Sonntag,3,53,1,1,1,0,0,0,0,...,0.23,0.0,5.9,0.0,0,0,0,0,0,0
3,Montag,4,1,0,0,0,0,0,0,0,...,0.13,0.3,2.4,0.0,0,0,0,0,0,0
4,Dienstag,5,1,0,0,0,0,0,0,0,...,0.00,0.0,3.4,1.3,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334,Dienstag,1335,35,0,0,0,0,0,0,0,...,0.00,0.0,11.2,0.0,2,0,59,26,29,114
1335,Mittwoch,1336,35,0,0,0,0,0,0,0,...,0.00,0.0,5.6,1.1,2,0,61,40,19,120
1336,Donnerstag,1337,35,0,0,0,0,0,0,0,...,0.16,0.0,5.8,0.0,2,0,76,38,23,137
1337,Freitag,1338,35,0,0,0,0,0,0,0,...,0.00,0.0,5.7,0.7,3,0,157,48,27,232


In [173]:
# Create a boolean mask for categorical columns 
categorical_mask = (df.dtypes == object)  # true → is categorical
# Get list of categorical column names
categorical_columns = df.columns[categorical_mask].tolist()
# Print the head of the categorical columns 
print(df[categorical_columns].head())
# Create LabelEncoder object: le 
le = LabelEncoder()
# Apply LabelEncoder to categorical columns
df[categorical_columns] = df[categorical_columns].apply(lambda x: le.fit_transform(x))
# Print the head of the LabelEncoded categorical columns print(df[categorical_columns].head())

  Wochentag Ferien Desc ZH Ferien Desc SG Ferien Desc GR  Feiertag Desc
0   Freitag              0              0              0        Neujahr
1   Samstag              0              0              0  Berchtoldstag
2   Sonntag              0              0              0              0
3    Montag              0              0              0              0
4  Dienstag              0              0              0              0


## One Hot Encoding

In [179]:
# Create OneHotEncoder: ohe
ohe = OneHotEncoder(categorical_features=categorical_mask, sparse=False)
# Apply OneHotEncoder to categorical columns - output is no longer a dataframe: df_encoded df_encoded = ohe.fit_transform(df)
# Print first 5 rows of the resulting dataset - again, this will no longer be a pandas dataframe print(df_encoded[:5, :])
# Print the shape of the original DataFrame 
d_encoded = ohe.fit_transform(df)
print(df.shape)
# Print the shape of the transformed array 
print(d_encoded.shape)
print(type(d_encoded))
d_encoded

(1339, 53)
(1339, 111)
<class 'numpy.ndarray'>


array([[  0.,   0.,   1., ...,  20.,   5.,  35.],
       [  0.,   0.,   0., ...,  59.,  22., 159.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [  0.,   1.,   0., ...,  38.,  23., 137.],
       [  0.,   0.,   1., ...,  48.,  27., 232.],
       [  0.,   0.,   0., ...,  33.,  23., 239.]])

## Modelling

In [180]:
target_variable = 'Schalter'
col_names= list(df)
col_names.remove(target_variable)
col_names.remove('Tel')
col_names.remove('Mail')
col_names.remove('Total Anfragen')
#col_names.remove('Datum')
print(len(col_names))

print(d_encoded[:,:-4].shape)

49
(1339, 107)


In [184]:
X, y = d_encoded[:,:-4],d_encoded[:,-4]
print(X.shape)
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, 
                                                        random_state=122, shuffle=False)
DM_train = xgb.DMatrix(data=X_train,label=y_train)
DM_test =  xgb.DMatrix(data=X_test,label=y_test)

(1339, 107)


In [185]:
params = {"booster":"gblinear","objective":"reg:linear"}
xg_reg = xgb.train(params = params, dtrain=DM_train, num_boost_round=10)

preds = xg_reg.predict(DM_test)

rmse = np.sqrt(mean_squared_error(y_test,preds))

print("RMSE: %f" % (rmse))

[09:45:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE: 41.525184


In [182]:
def train_model(max_depth, 
                n_estimators,
                learning_rate):
    params = {
        'n_estimators': int(n_estimators),
        'max_depth': int(max_depth),
        'learning_rate':learning_rate,
        'n_jobs': 8, #number of CPU threads
        "booster":"gblinear",
        "objective":"reg:squarederror"
    }

#     pipeline = Pipeline([
#                      ("vectorizer", DictVectorizer(sort=False)),
#                      ("clf", xgb.XGBRegressor(objective = 'reg:squarederror',**params)),

#                         ])
    
#     xgb_fit = pipeline.fit(DM_train)
#     preds = xgb_fit.predict(DM_test)
#     mse = mean_absolute_error(preds, y_test)

    xg_reg = xgb.train(params = params, dtrain=DM_train, num_boost_round=10)
    preds = xg_reg.predict(DM_test)
    mse = mean_absolute_error(preds, y_test)
    return -mse

bounds = {
    'max_depth':(5,100),
    'n_estimators': (100,10000),
    'learning_rate': (.001, 0.1),
}

In [183]:
optimizer = BayesianOptimization(
    f=train_model,
    pbounds=bounds,
    random_state=1,
    verbose=2,
)
best = optimizer.maximize(init_points=10, n_iter= 10)
print(optimizer.max)

|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------
[09:37:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
|  1        | -23.82    |  0.04229  |  73.43    |  101.1    |
[09:37:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
|  2        | -24.39    |  0.03093  |  18.94    |  1.014e+0 |
[09:37:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
|  3        | -25.51    |  0.01944  |  37.83    |  4.028e+0 |
[09:37:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
|  4        | -23.55    |  0.05434  |  44.82    |  6.884e+0 |
[09:37:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
|  5        | -25.27    |  0.02124  |  88.42    